In [1]:
import fitz 
from sentence_transformers import SentenceTransformer
import psycopg2
import numpy as np

/Users/gizemkaryagdi/Library/Python/3.9/lib/python/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


PDF Yükleme

In [2]:
def extract_text_from_pdf(pdf_path):
    doc = fitz.open(pdf_path)
    text = ""
    for page_num in range(len(doc)):
        page = doc.load_page(page_num)
        text += page.get_text()
    return text

Vektör Gömme

In [3]:
def generate_embeddings(text, model_name='all-MiniLM-L6-v2'):
    model = SentenceTransformer(model_name)
    embeddings = model.encode(text)
    return embeddings

Gömmeleri PostgreSQL ile Saklama

In [4]:
def store_embeddings_in_db(embeddings, db_config):
    conn = psycopg2.connect(**db_config)
    cur = conn.cursor()
    
    # Tablo oluşturma
    cur.execute("""
    CREATE TABLE IF NOT EXISTS embeddings (
        id SERIAL PRIMARY KEY,
        embedding FLOAT8[]
    )
    """)
    
    # Veri tabanına ekleme
    for emb in embeddings:
        cur.execute("INSERT INTO embeddings (embedding) VALUES (%s)", (emb.tolist(),))
    
    conn.commit()
    cur.close()
    conn.close()

# Main function to execute the steps
def main(pdf_path, db_config):
    text = extract_text_from_pdf(pdf_path)
    embeddings = generate_embeddings(text.split('\n'))
    store_embeddings_in_db(embeddings, db_config)

db_config = {
    'dbname': 'postgres',
    'user': 'postgres',
    'password': '290128',
    'host': 'localhost',
    'port': 5432
}
pdf_path = '/Users/gizemkaryagdi/Desktop/MachineLearning-Lecture01.pdf'
main(pdf_path, db_config)